<a href="https://colab.research.google.com/github/labra/RailDataForum_ValidationWorkshop/blob/main/RailDataForumExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rail data forum - Validation workshop examples

In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 7.7 MB/s eta 0:00:00


In [2]:
from rdflib import Graph

In [3]:
g = Graph()

In [4]:
person_ttl_str = """
prefix : <http://example.org/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

:timbl a :Human ;
       :name       "Tim Berners-Lee" ;
       :birthPlace :london ;
       :birthDate  "1955-06-08"^^xsd:date ;
       :employer   :CERN ;
       :knows     :jose .

:london a :City, :Metropolis ;
        :country :UK .

:CERN a :Organization ;
      :name "CERN"    .

:jose :name       "Jose" ;
      :birthPlace :Oviedo .

:Oviedo a :City ;
        :country :Spain .
""";

In [5]:
g.parse(data=person_ttl_str, format="ttl")

<Graph identifier=N0ed6d325b13c4bd7ae6f65758d21142e (<class 'rdflib.graph.Graph'>)>

In [6]:
query_str = """
PREFIX : <http://example.org/>

SELECT ?person ?name ?country WHERE {
  ?person a :Human ;
    :name ?name ;
    :birthDate ?birthDate ;
    :birthPlace ?birthPlace .
  ?birthPlace :country ?country
}
""";

Run a SPARQL query

In [9]:
result = g.query(query_str);

In [10]:
for row in result:
    print(row)

(rdflib.term.URIRef('http://example.org/timbl'), rdflib.term.Literal('Tim Berners-Lee'), rdflib.term.URIRef('http://example.org/UK'))


## Validating with SHACL